# Klusterointi ja assosiaatio

## Tavoitteet
Tavoite tässä prosessissa on optimoida drooneja käyttävän kuljetusyrityksen toimintaa löytämällä optimaalisia sijainteja droonikeskuksille. Yritys haluaa myös suositella asiakkaille tuotteita heidän ostoshistoriansa perusteella, joten datasta halutaan löytää kiinnostavia yhteyksiä eri tuoteryhmien välillä. 

## Datan kuvaus
Käytössä on 2 eri datasettiä: 
- Asiakkaiden sijannit, joka pitää sisällään seuraavat sarakkeet:
- - clientid: Asiakkaan oma tunniste
- - x: Asiakkaan sijainnin x-kordinaatti
- - y: Asiakkaan sijainnin y-kordinaatti
- Tuotteiden myyntidata, joka pitää sisällään seuraavat sarakkeet:
- - ID: Ostotapahtuman tunniste
- - Prod1-Prod20: Binääriarvo, joka kertoo ostettiinko tietystä tuoteryhmästä tuote vai ei.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df_cust = pd.read_csv('drone_cust_locations.csv')
df_prod = pd.read_csv('drone_prod_locations.csv')

## Datan esikäsittely

## Mallinnus

## Arviointi

## Käyttöönotto
Yritys voisi käyttää käsittelystä johdettuja optimaalisia droonikeskuksen sijainteja uusien droonikeskusten sijaintien valintaan. Tuotteiden ostotapahtumien väliset yhteydet voisi suoraan implementoida yrityksen verkkosivuille/sovellukseen antamaan asiakkaille suosituksia, kun hän selailee kauppaa. 